<a href="https://colab.research.google.com/github/bartianbosch/Melk/blob/Jinli/Big_Data_Milk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Big Data Groupwork Milk


In [ ]:
# To download and update the necessary requirements, run this cell
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://downloads.apache.org/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
!tar xf spark-2.4.7-bin-hadoop2.7.tgz
!pip install -q findspark

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [443 kB]
Get:

In [ ]:
# To initialize Apache Spark, run this cell
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark.sql import SparkSession
sp = SparkSession.builder.master("local[*]").getOrCreate()

from pyspark import SparkContext
from pyspark.sql import SparkSession
sc = SparkContext.getOrCreate()
spark = SparkSession.builder.getOrCreate()

from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


Loading the data into a DF


In [ ]:
!ls 'gdrive/MyDrive/'

 Big_Data_Groupwork   Big_Data_Groupwork_personal  'Colab Notebooks'   INF33806


In [ ]:
# try one of these (for now)


milk_rdd = spark.read.csv('/gdrive/MyDrive/Big_Data_Groupwork/ff_dairy_v1.csv', header=True, inferSchema = True)
#milk_data = spark.read.csv('/content/gdrive/MyDrive/Big_Data_Groupwork_Project/Big_Data_Groupwork/ff_dairy_v1.csv', header=True, inferSchema = True)

# work from dataframes is sufficient

AnalysisException: ignored

In [ ]:
milk_data.show(5)

+----------+-------+----+---------+---------+----------------+------------+----------+----+------------------------------------------+-----------------------------------------------+---------------------------+---------------------------+-----------------+-----------------------+-----------+----+-----------------+---------------+----+--------+------------+--------------+--------+---------------------+---------+---------------------+-------------------------+-------+-----------------------+---------------+-------------------------------------+--------------------------------------+-----------------+-------------------------+-------------------+-----------------------+-------+-------------------------+--------------------------+--------------+------------------+------------------+----------------------+------------------+--------+-------------------+-------------+---------------------+-----------------+--------------+----------+---------+----------+----------+---------+----------+-------

Qusetion 1: 

Which combination of nutrients is the best predictor for the fat yield (milk yield * fat %) ?

In [ ]:
# select the yield column and fat% column

yield_col = milk_data.select('Milk yield (kg)')
fat_pc_col = milk_data.select('Fat%')

yield_col.take(5)
yield_col.show(5)

yield_fat = map(lambda x, y : x*y, (yield_col, fat_pc_col))
yield_fat.alias('Fat yield')



+---------------+
|Milk yield (kg)|
+---------------+
|          40.47|
|          38.29|
|          39.28|
|          38.16|
|          44.06|
+---------------+
only showing top 5 rows



AttributeError: ignored

In [ ]:
# seletc the nutrient columns in g/day

# crude fiber, sugar, crude protein, crude fat, starch, Ca, P, Na, Cl, K, Mg, WDS, SPK, TPK, SPE, TPE, +I, Wi, GP 
# intestines: nXP, RNB, MELK, NEL, WDE, EKB

cr_fiber_col = milk_data.select('Crude fiber (g/day)')
sugar_col = milk_data.select('Sugar (g/day)')
cr_protein_col = milk_data.select('Crude protein (g/day)')
cr_fat_col = milk_data.select('Crude fat (g/day)')
starch_col = milk_data.select('Starch (g/day)')

ca_col = milk_data.select('Ca (g/day)')
p_col = milk_data.select('P (g/day)')
na_col = milk_data.select('Na (g/day)')
cl_col = milk_data.select('Cl (g/day)')
k_col = milk_data.select('K (g/day)')
mg_col = milk_data.select('Mg (g/day)')



In [ ]:
# create linear regression models 

import seaborn as sns 

sns.scatterplot(x = milk_data.select(*), y = milk_data.select(*))

SyntaxError: ignored